# 📘 Anleitung: Starten des Notebooks

Dieses Notebook wird von GitHub geladen. Um alle Funktionen zu aktivieren, führen Sie bitte folgende Schritte aus:

1. **Alles ausführen:** Gehen Sie oben im Menü auf **Laufzeit** → **Alle ausführen**.
2. **Warnung bestätigen:** Wenn das Fenster *"Warnung: Dieses Notebook wurde nicht von Google erstellt"* erscheint, klicken Sie auf **„Trotzdem ausführen“**.

---
*Die Warnung erscheint automatisch für alle Dateien von externen Quellen (GitHub).*

In [2]:
# UI (Button) – für eine "schöne" Ausführung direkt im Notebook
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

title = widgets.HTML(
    value="""
    <div style="padding:14px;border-radius:12px;border:1px solid #e5e7eb;background:#fafafa">
      <div style="font-size:18px;font-weight:700;margin-bottom:6px">LOTTO 6aus49 – Colab Runner</div>
      <div style="color:#374151;font-size:13px;line-height:1.45">
        Klicke auf <b>▶ Ausführen</b>, um Daten zu laden, das Modell zu trainieren, Zahlen vorherzusagen und ans Backend zu senden.
      </div>
    </div>
    """
)

btn_run = widgets.Button(
    description="▶ Ausführen",
    button_style="success",
    tooltip="Startet den kompletten Lauf",
    layout=widgets.Layout(width="180px", height="40px")
)

status = widgets.HTML(value="")
out = widgets.Output(layout={"border":"1px solid #e5e7eb","border_radius":"12px","padding":"10px"})
progress = widgets.IntProgress(value=0, min=0, max=100, description="Fortschritt:", bar_style="info")

display(title, widgets.HBox([btn_run, progress]), status, out)


HTML(value='\n    <div style="padding:14px;border-radius:12px;border:1px solid #e5e7eb;background:#fafafa">\n …

HTML(value='')

Output(layout=Layout(border='1px solid #e5e7eb', padding='10px'))

In [3]:
# Imports
import os
import io
import time
import json
import zipfile
import requests
from datetime import datetime

import numpy as np
import pandas as pd
import pytz

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam

In [4]:
# Konfiguration (Backend)
# Hinweis: "origin" kann optional per URL-Parameter übergeben werden:
# https://colab.research.google.com/github/.../LOTTO_6aus49_Datei.ipynb?origin=https://example.com
#
# In Colab können wir URL-Parameter oft über JavaScript auslesen.
# Falls das nicht klappt, verwenden wir BACKEND_URL_DEFAULT.

BACKEND_URL_DEFAULT = "https://apilotto.euroceiling39.ru"

# Modellparameter (wie im Original)
model_params = {
    "batch_size": 98,
    "epochs": 123,
    "window_length": 24
}

# ----------------------------
# Hilfsfunktionen
# ----------------------------
def ui_set_status(html, kind="info"):
    # kind: info/success/warn/error
    colors = {
        "info":   ("#1f2937", "#eef2ff", "#c7d2fe"),
        "success":("#065f46", "#ecfdf5", "#a7f3d0"),
        "warn":   ("#92400e", "#fffbeb", "#fcd34d"),
        "error":  ("#991b1b", "#fef2f2", "#fecaca"),
    }
    fg, bg, bd = colors.get(kind, colors["info"])
    status.value = f"""
    <div style="margin-top:10px;padding:10px 12px;border-radius:12px;
                border:1px solid {bd}; background:{bg}; color:{fg}; font-size:13px">
      {html}
    </div>
    """

def download_and_parse_lotto():
    # URL zum Herunterladen des Archivs
    url = "https://www.lotto-bayern.de/static/gamebroker_2/de/download_files/archiv_lotto.zip"

    tmp_dir = "/content/tmp_lotto"
    os.makedirs(tmp_dir, exist_ok=True)

    r = requests.get(url, timeout=60)
    if r.status_code != 200:
        raise RuntimeError(f"Download fehlgeschlagen. Status: {r.status_code}")

    # ZIP im Speicher öffnen und entpacken
    with zipfile.ZipFile(io.BytesIO(r.content)) as z:
        z.extractall(tmp_dir)

    txt_path = os.path.join(tmp_dir, "lotto.txt")
    if not os.path.exists(txt_path):
        raise RuntimeError("lotto.txt nicht gefunden (ZIP-Inhalt hat sich evtl. geändert).")

    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    rows = []
    for line in lines[1:]:  # Kopfzeile überspringen
        parts = line.split()
        if len(parts) < 10:
            continue

        tag, monat, jahr = parts[0], parts[1], parts[2]
        rest = parts[3:]

        dt = datetime(int(jahr), int(monat), int(tag))
        if dt >= datetime(2013, 5, 4):
            datum = dt.strftime("%d.%m.%Y")
            rest7 = rest[:7]  # 6 Zahlen + Superzahl
            if len(rest7) == 7:
                rows.append([datum] + rest7)

    df = pd.DataFrame(rows, columns=["Datum","Zahl1","Zahl2","Zahl3","Zahl4","Zahl5","Zahl6","Superzahl"])

    for c in ["Zahl1","Zahl2","Zahl3","Zahl4","Zahl5","Zahl6","Superzahl"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")

    df = df.dropna().reset_index(drop=True)
    if df.empty:
        raise RuntimeError("Keine Daten nach Filterung gefunden.")
    return df

def preprocess_df(df):
    # Datum und Superzahl entfernen
    df_numbers = df.drop(["Datum", "Superzahl"], axis=1).copy()
    # Zeilenweise sortieren (wie im Original)
    df_sorted = df_numbers.apply(lambda row: pd.Series(sorted(row.astype(int).tolist())), axis=1)
    return df_sorted

def create_training_data(df_sorted, window_length):
    scaler = StandardScaler().fit(df_sorted)
    transformed_df = scaler.transform(df_sorted)

    n_rows = df_sorted.shape[0]
    n_feat = df_sorted.shape[1]

    X = np.empty([n_rows - window_length, window_length, n_feat], dtype=float)
    y = np.empty([n_rows - window_length, n_feat], dtype=float)

    for i in range(0, n_rows - window_length):
        X[i] = transformed_df[i : i + window_length, 0 : n_feat]
        y[i] = transformed_df[i + window_length : i + window_length + 1, 0 : n_feat]

    return X, y, scaler

def create_model(input_shape, learning_rate=0.001):
    model = Sequential()
    model.add(Bidirectional(LSTM(240, input_shape=input_shape, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(240, return_sequences=True)))
    model.add(Dropout(0.2))
    model.add(Bidirectional(LSTM(240, return_sequences=True)))
    model.add(Bidirectional(LSTM(240, return_sequences=False)))
    model.add(Dropout(0.2))

    # Unverändert aus dem Original
    model.add(Dense(49))
    model.add(Dense(input_shape[1]))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse", metrics=["accuracy"])
    return model

def predict_last_window(model, df_sorted, scaler, window_length):
    to_predict = df_sorted.tail(window_length)

    if to_predict.shape[0] < window_length:
        padding = np.zeros((window_length - to_predict.shape[0], to_predict.shape[1]))
        to_predict = np.vstack((padding, to_predict))
    else:
        to_predict = np.array(to_predict)

    scaled_to_predict = scaler.transform(to_predict)
    y_pred = model.predict(np.array([scaled_to_predict]), verbose=0)
    predicted = scaler.inverse_transform(y_pred).astype(int)[0]

    # Optional: Bereich 1..49
    predicted = np.clip(predicted, 1, 49)
    return predicted

def get_origin_from_url_or_default():
    # Versucht, origin aus den Colab-URL-Parametern zu lesen.
    # Wenn nicht möglich, Default verwenden.
    try:
        # JavaScript-Bridge: gibt Window.location.search zurück
        from google.colab import output
        js = """
        (function() {
          return window.location.search || "";
        })();
        """
        q = output.eval_js(js)  # z.B. "?origin=https%3A%2F%2Fexample.com"
        if q and "origin=" in q:
            # sehr einfacher Parser
            import urllib.parse as up
            params = up.parse_qs(q.lstrip("?"))
            origin = params.get("origin", [None])[0]
            if origin:
                return origin
    except Exception:
        pass

    return BACKEND_URL_DEFAULT

def send_to_backend(backend_url, payload):
    r = requests.post(f"{backend_url}/api/colab/update", json=payload, timeout=30)
    return r

# ----------------------------
# Runner (wird vom Button aufgerufen)
# ----------------------------
def run_pipeline(_=None):
    progress.value = 0
    with out:
        clear_output()

    ui_set_status("Starte…", "info")

    try:
        progress.value = 5
        ui_set_status("1/5 Lade Lotto-Daten (ZIP) und parse lotto.txt…", "info")
        df_raw = download_and_parse_lotto()

        with out:
            print("✅ Daten geladen:", df_raw.shape)
            display(df_raw.head())

        progress.value = 20
        ui_set_status("2/5 Vorverarbeitung und Trainingsdaten erstellen…", "info")
        df_sorted = preprocess_df(df_raw)
        X, y, scaler = create_training_data(df_sorted, model_params["window_length"])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.12, random_state=24)

        progress.value = 35
        ui_set_status("3/5 Modell erstellen…", "info")
        input_shape = (model_params["window_length"], X.shape[2])
        model = create_model(input_shape)

        model_params_runtime = dict(model_params)
        model_params_runtime.update({
            "layers": len(model.layers),
            "optimizer": str(model.optimizer),
            "loss": model.loss,
            "metrics": model.metrics_names,
        })

        progress.value = 45
        ui_set_status(f"4/5 Training läuft… (epochs={model_params_runtime['epochs']}, batch_size={model_params_runtime['batch_size']})", "warn")

        history = model.fit(
            x=X_train,
            y=y_train,
            batch_size=model_params_runtime["batch_size"],
            epochs=model_params_runtime["epochs"],
            verbose=2
        )

        progress.value = 80
        ui_set_status("Training fertig. Evaluiere…", "info")
        train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
        test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

        predicted_numbers = predict_last_window(model, df_sorted, scaler, model_params_runtime["window_length"])

        with out:
            print("\n📌 Metriken")
            print(f"Train Loss: {train_loss}, Train Accuracy: {train_accuracy}")
            print(f"Test  Loss: {test_loss},  Test  Accuracy: {test_accuracy}")
            print("\n🎯 Vorhergesagte Zahlen:", predicted_numbers)

        progress.value = 90
        ui_set_status("5/5 Sende Ergebnisse ans Backend…", "info")

        backend_url = get_origin_from_url_or_default()

        payload = {
            "numbers": predicted_numbers.tolist() if hasattr(predicted_numbers, "tolist") else list(predicted_numbers),
            "train": {"loss": float(train_loss), "accuracy": float(train_accuracy)},
            "test": {"loss": float(test_loss), "accuracy": float(test_accuracy)},
            "model_params": model_params_runtime,
            "generated_at": int(time.time()),
        }

        resp = send_to_backend(backend_url, payload)

        if resp.ok:
            ui_set_status(f"✅ Gesendet! Backend: <code>{backend_url}</code><br>Antwort: {resp.status_code}", "success")
            progress.value = 100
        else:
            ui_set_status(f"❌ Fehler beim Senden: {resp.status_code}<br><pre style='white-space:pre-wrap'>{resp.text[:800]}</pre>", "error")
            progress.value = 100

    except Exception as e:
        ui_set_status(f"⚠️ Lauf fehlgeschlagen: <code>{str(e)}</code>", "error")
        progress.value = 100

btn_run.on_click(run_pipeline)
ui_set_status("Bereit. Klicke auf <b>▶ Ausführen</b>.", "info")
